# <div align="center" style="color: brown"><strong>Polynomial Regression Tutorial</strong></div>

## <div style="color: red"><strong>Part 1. Introduction to Polynomial Regression</strong></div>

Polynomial regression is an extension of linear regression that allows us to model non-linear relationships between variables. While linear regression fits a straight line to data, polynomial regression fits a curve by including polynomial terms (squared, cubed, etc.) of the independent variable.

### Mathematical Representation

The mathematical equation for polynomial regression is:

$$Y = β₀ + β₁X + β₂X^2 + β₃X^3 + ... + βₙX^n + ε$$

Where:
- Y is the dependent variable (what we're trying to predict)
- X is the independent variable
- X^2, X^3, ..., X^n are the polynomial terms of X
- β₀ is the y-intercept (constant term)
- β₁, β₂, ..., βₙ are the coefficients for each term
- ε is the error term (residual)
- n is the degree of the polynomial

### Key Differences from Linear Regression

- **Non-linear relationships**: Can model curved relationships between variables
- **Flexibility**: Higher degree polynomials can capture more complex patterns in data
- **Overfitting risk**: Higher degree polynomials may fit training data too closely, leading to poor generalization

### When to Use Polynomial Regression

1. When there's a clear non-linear pattern in your data
2. When a simple linear model doesn't adequately capture the relationship
3. When you need to model curved relationships, peaks, or valleys in your data

## <div style="color: red"><strong>Part 2. Implementing Polynomial Regression in Python</strong></div>

Let's start by importing the necessary libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline

# Set the style for our plots
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")

### 2.1 Creating a Synthetic Dataset

Let's create a synthetic dataset that demonstrates a non-linear relationship between variables. We'll create data that follows a cubic polynomial pattern with some added noise.

In [ ]:
# Set a random seed for reproducibility
np.random.seed(42)

# Generate 100 points between -10 and 10
n_samples = 100
X = np.linspace(-10, 10, n_samples).reshape(-1, 1)

# Generate target variable following a cubic polynomial: y = 1 + 2x + 3x² - 0.5x³ + noise
true_coef = [1, 2, 3, -0.5]  # Coefficients: constant, x, x², x³
y_true = true_coef[0] + true_coef[1]*X.flatten() + true_coef[2]*X.flatten()**2 + true_coef[3]*X.flatten()**3
y = y_true + np.random.normal(0, 20, n_samples)

# Create a DataFrame
data = pd.DataFrame({
    'X': X.flatten(),
    'y': y
})

# Display the first few rows
data.head()

### 2.2 Visualizing the Dataset

Let's visualize our dataset to see the non-linear pattern:

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['X'], data['y'], alpha=0.7)
plt.xlabel('X', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.title('Non-linear Relationship Between X and y', fontsize=16)
plt.grid(True)
plt.show()

## <div style="color: red"><strong>Part 3. Comparing Linear and Polynomial Models</strong></div>

Let's first fit a simple linear regression model to see how it performs on our non-linear data. Then we'll compare it with polynomial regression models of different degrees.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit a linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Create polynomial regression models with different degrees
models = {
    'Linear': make_pipeline(LinearRegression()),
    'Quadratic': make_pipeline(PolynomialFeatures(degree=2, include_bias=False), LinearRegression()),
    'Cubic': make_pipeline(PolynomialFeatures(degree=3, include_bias=False), LinearRegression()),
    'Quartic': make_pipeline(PolynomialFeatures(degree=4, include_bias=False), LinearRegression()),
    'Degree 5': make_pipeline(PolynomialFeatures(degree=5, include_bias=False), LinearRegression()),
    'Degree 10': make_pipeline(PolynomialFeatures(degree=10, include_bias=False), LinearRegression())
}

# Fit all models
for name, model in models.items():
    model.fit(X_train, y_train)

# Predict on test data
predictions = {}
scores = {}

for name, model in models.items():
    predictions[name] = model.predict(X_test)
    scores[name] = {
        'R² Score': r2_score(y_test, predictions[name]),
        'RMSE': np.sqrt(mean_squared_error(y_test, predictions[name]))
    }
    
# Display performance metrics
performance = pd.DataFrame(scores).T
print("Model Performance on Test Data:")
performance

In [ ]:
# Visualize the fitted models
# Create a smooth grid of points for plotting
X_grid = np.linspace(X.min(), X.max(), 1000).reshape(-1, 1)

# Plot data and models
plt.figure(figsize=(12, 8))

# Plot the data points
plt.scatter(X, y, color='blue', alpha=0.5, label='Data Points')

# Plot the true curve
y_true_grid = true_coef[0] + true_coef[1]*X_grid.flatten() + true_coef[2]*X_grid.flatten()**2 + true_coef[3]*X_grid.flatten()**3
plt.plot(X_grid, y_true_grid, 'k-', linewidth=2.5, label='True Relationship')

# Plot the predictions for each model
colors = ['red', 'green', 'purple', 'orange', 'brown', 'magenta']
i = 0
for name, model in models.items():
    y_grid_pred = model.predict(X_grid)
    plt.plot(X_grid, y_grid_pred, color=colors[i], linewidth=2, label=f'{name} Model')
    i += 1

plt.title('Comparison of Polynomial Regression Models', fontsize=16)
plt.xlabel('X', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.legend()
plt.grid(True)
plt.show()

## <div style="color: red"><strong>Part 4. Understanding Polynomial Regression Coefficients</strong></div>

In polynomial regression, we have coefficients for each power of X. Let's extract and interpret these coefficients from our models.

In [ ]:
# Extract coefficients from the cubic model (degree 3)
cubic_model = models['Cubic']
poly_features = cubic_model.named_steps['polynomialfeatures']
linear_reg = cubic_model.named_steps['linearregression']

# Get feature names for the polynomial terms
feature_names = poly_features.get_feature_names_out(['X'])

# Create DataFrame with coefficients
coef_df = pd.DataFrame({
    'Term': feature_names,
    'Coefficient': linear_reg.coef_
})

# Add intercept
coef_df = pd.concat([pd.DataFrame({'Term': ['Intercept'], 'Coefficient': [linear_reg.intercept_]}), coef_df])

print("Cubic Model Coefficients:")
coef_df

## <div style="color: red"><strong>Part 5. Residual Analysis</strong></div>

Residual analysis helps us evaluate model performance and check if our assumptions are met. Let's analyze the residuals for our linear and cubic models.

In [ ]:
# Calculate residuals for linear and cubic models
linear_residuals = y_test - predictions['Linear']
cubic_residuals = y_test - predictions['Cubic']

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot residuals vs. predicted values for linear model
axes[0, 0].scatter(predictions['Linear'], linear_residuals)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_title('Linear Model: Residuals vs Predicted', fontsize=14)
axes[0, 0].set_xlabel('Predicted values', fontsize=12)
axes[0, 0].set_ylabel('Residuals', fontsize=12)
axes[0, 0].grid(True)

# Plot residuals vs. predicted values for cubic model
axes[0, 1].scatter(predictions['Cubic'], cubic_residuals)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_title('Cubic Model: Residuals vs Predicted', fontsize=14)
axes[0, 1].set_xlabel('Predicted values', fontsize=12)
axes[0, 1].set_ylabel('Residuals', fontsize=12)
axes[0, 1].grid(True)

# Histogram of residuals for linear model
axes[1, 0].hist(linear_residuals, bins=15, edgecolor='black')
axes[1, 0].set_title('Linear Model: Residuals Distribution', fontsize=14)
axes[1, 0].set_xlabel('Residual Value', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].grid(True)

# Histogram of residuals for cubic model
axes[1, 1].hist(cubic_residuals, bins=15, edgecolor='black')
axes[1, 1].set_title('Cubic Model: Residuals Distribution', fontsize=14)
axes[1, 1].set_xlabel('Residual Value', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## <div style="color: red"><strong>Part 6. Understanding Overfitting in Polynomial Regression</strong></div>

While higher-degree polynomials can better fit training data, they risk overfitting, which means they perform poorly on new data. Let's demonstrate this concept.

In [ ]:
# Create a smaller dataset to emphasize overfitting
np.random.seed(42)
n_samples_small = 15
X_small = np.linspace(-10, 10, n_samples_small).reshape(-1, 1)
y_true_small = true_coef[0] + true_coef[1]*X_small.flatten() + \
               true_coef[2]*X_small.flatten()**2 + true_coef[3]*X_small.flatten()**3
y_small = y_true_small + np.random.normal(0, 20, n_samples_small)

# Create models with different degrees
degrees = [1, 3, 10, 20]
models_small = {}

for degree in degrees:
    model_name = f'Degree {degree}'
    models_small[model_name] = make_pipeline(PolynomialFeatures(degree=degree), LinearRegression())
    models_small[model_name].fit(X_small, y_small)

# Create a smooth grid for plotting
X_grid = np.linspace(X_small.min(), X_small.max(), 1000).reshape(-1, 1)

# Plot the results
plt.figure(figsize=(12, 8))

# Plot the true curve
y_true_grid = true_coef[0] + true_coef[1]*X_grid.flatten() + \
              true_coef[2]*X_grid.flatten()**2 + true_coef[3]*X_grid.flatten()**3
plt.plot(X_grid, y_true_grid, 'k-', linewidth=2.5, label='True Relationship')

# Plot the data points
plt.scatter(X_small, y_small, color='blue', s=60, alpha=0.7, label='Data Points')

# Plot each model
colors = ['red', 'green', 'purple', 'orange']
for i, (name, model) in enumerate(models_small.items()):
    y_pred = model.predict(X_grid)
    plt.plot(X_grid, y_pred, color=colors[i], linewidth=2, label=f'{name}')

plt.title('Overfitting in Polynomial Regression', fontsize=16)
plt.xlabel('X', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# Calculate training and test errors for different polynomial degrees
degrees = list(range(1, 21))  # Degrees from 1 to 20
train_errors = []
test_errors = []

# Generate a larger dataset for more reliable testing
np.random.seed(42)
n_samples_large = 500
X_large = np.linspace(-10, 10, n_samples_large).reshape(-1, 1)
y_true_large = true_coef[0] + true_coef[1]*X_large.flatten() + \
               true_coef[2]*X_large.flatten()**2 + true_coef[3]*X_large.flatten()**3
y_large = y_true_large + np.random.normal(0, 20, n_samples_large)

# Split data into training and testing sets
X_train_large, X_test_large, y_train_large, y_test_large = train_test_split(
    X_large, y_large, test_size=0.3, random_state=42)

# Calculate errors for each degree
for degree in degrees:
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train_large)
    X_test_poly = poly.transform(X_test_large)
    
    # Fit model
    model = LinearRegression()
    model.fit(X_train_poly, y_train_large)
    
    # Calculate errors
    train_pred = model.predict(X_train_poly)
    test_pred = model.predict(X_test_poly)
    
    train_mse = mean_squared_error(y_train_large, train_pred)
    test_mse = mean_squared_error(y_test_large, test_pred)
    
    train_errors.append(np.sqrt(train_mse))  # RMSE
    test_errors.append(np.sqrt(test_mse))    # RMSE

# Plot the errors
plt.figure(figsize=(10, 6))
plt.plot(degrees, train_errors, marker='o', linestyle='-', color='blue', label='Training RMSE')
plt.plot(degrees, test_errors, marker='o', linestyle='-', color='red', label='Test RMSE')

# Add vertical line at degree 3 (our true model)
plt.axvline(x=3, color='green', linestyle='--', label='True Model Degree (3)')

plt.title('Training vs. Test Error for Different Polynomial Degrees', fontsize=16)
plt.xlabel('Polynomial Degree', fontsize=14)
plt.ylabel('Root Mean Squared Error (RMSE)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

## <div style="color: red"><strong>Part 7. Regularization to Prevent Overfitting</strong></div>

To prevent overfitting when using high-degree polynomials, we can apply regularization techniques like Ridge or Lasso regression.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline

# Define models with different regularization techniques
degree = 10  # High degree polynomial
models_reg = {
    'No Regularization': make_pipeline(
        PolynomialFeatures(degree=degree),
        LinearRegression()
    ),
    'Ridge (alpha=1)': make_pipeline(
        PolynomialFeatures(degree=degree),
        Ridge(alpha=1)
    ),
    'Ridge (alpha=10)': make_pipeline(
        PolynomialFeatures(degree=degree),
        Ridge(alpha=10)
    ),
    'Lasso (alpha=0.1)': make_pipeline(
        PolynomialFeatures(degree=degree),
        Lasso(alpha=0.1, max_iter=10000)
    ),
    'Lasso (alpha=1)': make_pipeline(
        PolynomialFeatures(degree=degree),
        Lasso(alpha=1, max_iter=10000)
    )
}

# Fit all models on the small dataset to emphasize overfitting
for name, model in models_reg.items():
    model.fit(X_small, y_small)

# Create predictions on a grid
y_preds_reg = {}
for name, model in models_reg.items():
    y_preds_reg[name] = model.predict(X_grid)

# Plot the results
plt.figure(figsize=(12, 8))

# Plot the true curve
plt.plot(X_grid, y_true_grid, 'k-', linewidth=2.5, label='True Relationship')

# Plot the data points
plt.scatter(X_small, y_small, color='blue', s=60, alpha=0.7, label='Data Points')

# Plot each model
colors = ['red', 'green', 'purple', 'orange', 'magenta']
for i, (name, y_pred) in enumerate(y_preds_reg.items()):
    plt.plot(X_grid, y_pred, color=colors[i], linewidth=2, label=f'{name}')

plt.title(f'Regularized Polynomial Regression (Degree {degree})', fontsize=16)
plt.xlabel('X', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

## <div style="color: red"><strong>Part 8. Real-World Application Example</strong></div>

Let's apply polynomial regression to a slightly more realistic scenario. We'll create a dataset that mimics the relationship between advertising spending and sales.

In [ ]:
# Create synthetic advertising-sales data
np.random.seed(42)

# Create advertising spending data (in thousands of dollars)
advertising = np.random.uniform(0, 50, 100).reshape(-1, 1)

# Create sales data with diminishing returns (in thousands of dollars)
# Formula: sales = 10 + 3*advertising - 0.03*advertising^2 + noise
sales = 10 + 3 * advertising.flatten() - 0.03 * advertising.flatten()**2 + np.random.normal(0, 5, 100)

# Create a DataFrame
ad_data = pd.DataFrame({
    'Advertising': advertising.flatten(),
    'Sales': sales
})

# Show the first few rows
ad_data.head()

In [ ]:
# Visualize the data
plt.figure(figsize=(10, 6))
plt.scatter(ad_data['Advertising'], ad_data['Sales'], alpha=0.7)
plt.xlabel('Advertising Spend (thousands $)', fontsize=14)
plt.ylabel('Sales (thousands $)', fontsize=14)
plt.title('Relationship Between Advertising and Sales', fontsize=16)
plt.grid(True)
plt.show()

# Split data into training and testing sets
X_ad = ad_data[['Advertising']].values
y_ad = ad_data['Sales'].values
X_ad_train, X_ad_test, y_ad_train, y_ad_test = train_test_split(X_ad, y_ad, test_size=0.2, random_state=42)

# Create and fit models with different degrees
ad_models = {
    'Linear': make_pipeline(PolynomialFeatures(degree=1), LinearRegression()),
    'Quadratic': make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    'Cubic': make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
}

for name, model in ad_models.items():
    model.fit(X_ad_train, y_ad_train)

# Create a smooth grid for visualization
X_ad_grid = np.linspace(X_ad.min(), X_ad.max(), 1000).reshape(-1, 1)

# Make predictions
ad_predictions = {}
ad_scores = {}

for name, model in ad_models.items():
    y_pred = model.predict(X_ad_test)
    ad_scores[name] = {
        'R²': r2_score(y_ad_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_ad_test, y_pred))
    }
    ad_predictions[name] = model.predict(X_ad_grid)

# Display metrics
ad_performance = pd.DataFrame(ad_scores).T
print("Model Performance on Test Data:")
ad_performance

In [ ]:
# Plot the results
plt.figure(figsize=(12, 8))

# Plot the data points
plt.scatter(X_ad, y_ad, color='blue', alpha=0.6, label='Data Points')

# Plot the model predictions
colors = ['red', 'green', 'purple']
for i, (name, y_pred) in enumerate(ad_predictions.items()):
    plt.plot(X_ad_grid, y_pred, color=colors[i], linewidth=2, label=f'{name} Model')

plt.title('Advertising-Sales Relationship: Model Comparison', fontsize=16)
plt.xlabel('Advertising Spend (thousands $)', fontsize=14)
plt.ylabel('Sales (thousands $)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# Analyze the best model (Quadratic) to find the point of diminishing returns
quadratic_model = ad_models['Quadratic']

# Extract coefficients from the quadratic model
poly = quadratic_model.named_steps['polynomialfeatures']
linreg = quadratic_model.named_steps['linearregression']

# Coefficients: β₀ + β₁x + β₂x²
intercept = linreg.intercept_
coef = linreg.coef_

print(f"Quadratic Model Equation: Sales = {intercept:.2f} + {coef[0]:.2f}×Advertising + {coef[1]:.2f}×Advertising²")

# Calculate derivative: dy/dx = β₁ + 2×β₂x
# At maximum/minimum point, dy/dx = 0, so x = -β₁/(2×β₂)
if coef[1] != 0:  # Ensure we don't divide by zero
    optimal_advertising = -coef[0] / (2 * coef[1])
    
    if optimal_advertising > 0 and optimal_advertising < X_ad.max():
        optimal_sales = intercept + coef[0] * optimal_advertising + coef[1] * optimal_advertising**2
        print(f"\nOptimum Advertising Spending: ${optimal_advertising:.2f} thousand")
        print(f"Estimated Maximum Sales: ${optimal_sales:.2f} thousand")
        
        # If it's a maximum (negative coefficient for x²)
        if coef[1] < 0:
            print("After this point, diminishing returns will reduce the effectiveness of additional advertising spend.")
        else:
            print("This is a minimum point. Sales increase at an increasing rate after this point.")
    else:
        print("\nOptimum point is outside the range of our data.")
else:
    print("\nThis is a linear relationship, not a curve with an optimum point.")

## <div style="color: red"><strong>Part 9. Summary and Conclusion</strong></div>

In this tutorial, we've covered:

1. **Theory of Polynomial Regression**
   - Mathematical representation and key concepts
   - Differences from linear regression
   - When to use polynomial regression

2. **Implementing Polynomial Regression in Python**
   - Creating and visualizing synthetic datasets
   - Training models with different polynomial degrees
   - Comparing model performance

3. **Advanced Concepts**
   - Understanding overfitting in polynomial regression
   - Regularization techniques to prevent overfitting
   - Analyzing a real-world application (advertising-sales relationship)

### Key Takeaways

- Polynomial regression allows us to model curved relationships between variables
- Higher degree polynomials provide more flexibility but risk overfitting
- Regularization techniques like Ridge and Lasso help prevent overfitting
- Testing multiple model degrees helps identify the best trade-off between bias and variance
- Real-world applications often show non-linear patterns that polynomial regression can capture effectively

### Next Steps

1. Apply polynomial regression to real datasets
2. Explore other non-linear regression techniques like splines and local regression
3. Learn about methods for automatic model selection
4. Study cross-validation techniques for more robust model evaluation
5. Consider other advanced regression techniques like Support Vector Regression or Gradient Boosting Regression